In [ ]:
!pip install tensorflow_text natasha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 97.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=547267eeb4501f1f8b2166d2a58e72165aba69ee82fa80ab42bf54b9e3494bb2
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26094 sha256=8598911074d9069aad5ae534d32a9c874c8a9c2a9f71c58a566ca348c708f578
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c8

In [ ]:
import re
import time

import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup  # Для удаления HTML тегов

import tensorflow as tf
from tensorflow.keras.saving import load_model
import tensorflow_hub as hub
import tensorflow_text as text

from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,
    Doc
)
from google.colab import drive
from matplotlib import pyplot as plt
import joblib

# Dataset_import

In [ ]:
# вход на файл
drive.mount("/content/drive")
DIR = '/content/drive/My Drive/Colab Notebooks/'
df = pd.read_excel(DIR + "CRA_train_1200.xlsx")

Mounted at /content/drive


# Models loading
<b style="color:red"> warning! it cat take a lot of time! </b>

In [ ]:
model_cat = load_model(DIR + 'model_cat_3.keras', custom_objects={'KerasLayer': hub.KerasLayer})
model_rat = load_model(DIR + 'model_rat_3.keras', custom_objects={'KerasLayer': hub.KerasLayer})

In [ ]:
le_cat = joblib.load(DIR + 'label_encoder_7.joblib')
le_rat = joblib.load(DIR + 'label_encoder_17.joblib')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Inicialize class

In [ ]:
class Nlp:
  def __init__(self):
        # Инициализация объектов, необходимых для обработки текста с использованием Natasha и Spacy.
        self.segmenter = Segmenter()  # Сегментация текста на предложения.
        self.morph_vocab = MorphVocab()  # Создание словаря морфологических данных.
        self.emb = NewsEmbedding()  # Получение векторных представлений слов.
        self.morph_tagger = NewsMorphTagger(self.emb)  # Морфологическая разметка текста.
        self.ner_tagger = NewsNERTagger(self.emb)  # Распознавание именованных сущностей.
        self.names_extractor = NamesExtractor(self.morph_vocab)  # Извлечение имен.
        self.stop_words = ['АО «Эксперт РА', 'АКРА', 'Компания', 'Группа', 'Эксперт РА', 'Рейтинговое агентство', 'АО Эксперт  РА', 'Кредитные', 'Оценка внешнее влияние', 'Группа.']


    # Очищаем текст регулярными выражениями
  def clear_text(self, text):
      soup = BeautifulSoup(text)
      text = soup.get_text()
      text = re.sub(r'(http\S+)|(www\S+)|([\w\d]+www\S+)|([\w\d]+http\S+)', '', text)
      text = re.sub(r'[\n\t\«]', ' ', text).strip()  # Перенос, табуляция
      text = re.sub(r'[^\w\d\s\.\,\"]', ' ', text)  # Только слова, цифры, пробелы, точки и запятые
      text = re.sub(r'\s+', ' ', text)  # Удаляем двойные пробелы
      pat = "\s+([{}]+)".format(re.escape("\.\,"))
      text = re.sub("\s{2,}", " ", re.sub(pat, r"\1", text))
      return text

  def get_features(self, text):
      doc = Doc(text)  # Создание объекта Doc для текста.
      doc.segment(self.segmenter)  # Сегментация текста на предложения.
      doc.tag_morph(self.morph_tagger)  # Морфологическая разметка текста.
      doc.tag_ner(self.ner_tagger)
      words = [token.text for token in doc.tokens if token.pos not in ["ADP", "PUNCT", "NUM", "CCONJ", 'PROPN']]
      count = len(words)  # Количество слов в строке
      if count > 0:
          average = sum(len(word) for word in words) / count
          uniq = round(100*len(set(words))/count)  # % уникальных слов в строке
      for span in doc.spans:
        span.normalize(self.morph_vocab)  # Нормализация именованных сущностей.
      ners = [(span.normal, span.type) for span in doc.spans if span.normal not in self.stop_words]
      counted_ners = self.ners_counter(ners)
      features_list = [count, average, uniq, counted_ners["ORG"], counted_ners["LOC"], counted_ners["PER"]]
      return features_list

  def ners_counter(self, ners):
      counted_ners = {"ORG": 0, "LOC": 0, "PER": 0}
      for ner in ners:
          counted_ners[ner[1]] += 1
      return counted_ners

  def prediction_pipeline(self, df):
    copied_df = df["pr_txt"].copy()
    cleared_text_list = list(copied_df.map(self.clear_text))
    df_text = pd.DataFrame(cleared_text_list, columns=['text'])
    x_features = np.array(list(df_text['text'].map(nlp.get_features)))
    df_features = pd.DataFrame(x_features, columns=['count', 'average', 'uniq', 'org', 'loc', 'per'])
    pred_cat_v = model_cat.predict([df_text, df_features])
    pred_rat_v = model_rat.predict([df_text, df_features])
    pred_cat_am = np.argmax(pred_cat_v, axis=1)
    pred_rat_am = np.argmax(pred_rat_v, axis=1)
    pred_cat_y = le_cat.inverse_transform(pred_cat_am)
    pred_rat_y = le_rat.inverse_transform(pred_rat_am)
    answer = pd.DataFrame({'Категория': pred_cat_y, 'Уровень рейтинга': pred_rat_y})
    out = pd.concat([copied_df, answer], axis=1)
    out.to_excel(DIR + 'submit.xlsx', index=False)

In [ ]:
nlp = Nlp()

In [ ]:
nlp.prediction_pipeline(df)